<img src="https://cdn.comet.ml/img/notebook_logo.png">

# Introduction
[TensorboardX](https://github.com/lanpa/tensorboardX) is an amazing project that enables users to leverage a Tensorboard like API to track and visualize their model metrics and parameters. 

[Comet](https://www.comet.ml/site/data-scientists/?utm_campaign=tensorboardx-integration&utm_medium=colab) is an MLOps Platform that is designed to help Data Scientists and Teams build better models faster! Comet provides tooling to track, Explain, Manage, and Monitor your models in a single place! It works with Jupyter Notebooks and Scripts and most importantely it's 100% free!

Comet now supports autologging straight from TensorboardX! In this guide, we will show you how easy it is to get started with visualizing assets logged through TensorboardX within Comet. 

# Setup 

---






## Install Dependencies

In [ ]:
%pip install comet_ml tensorboardX torch torchvision matplotlib

# Logging to Comet 

Using Comet with TensorboardX requires a very minimal change to your existing code. 

In order for Comet to log from TensorboardX, it must be imported before any ML Frameworks. 

In the snippet below, we will import Comet first, and initialize our Project Name. 

## Initialize Comet 

Initialize a Comet Project for logging our data. In this case, we're creating a project named "tensorboardX". You can set the project name to be anything you want.  

In [ ]:
import comet_ml

comet_ml.init(project_name="comet-example-pytorch-tensorboardX")

In [ ]:
# Helper function to display logged assets in the Comet UI
def display(tab=None):
    experiment = comet_ml.get_global_experiment()
    experiment.display(tab=tab)

In [ ]:
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime

## Initialize TensorboardX SummaryWriter with the Comet Flag

Assuming you have already imported Comet at the beginning of your script, all you have to do now is set the `comet_config` argument when initializing TensorboardX's SummaryWriter. 

`comet_config` is just a dictionary that allows you to set your workspace, project name and API key if you haven't already done so through `comet.init()`. 

The dictionary has the following fields
```
{
   "api_key":"Your Comet API Key",
   "workspace":"Your Comet Workspace Name",
   "project_name":"Your Comet Project Name",
   "disabled": False
}
```

By default, the `disabled` field is set to `True`. Change it to `False` to enable logging to Comet. 


In [ ]:
# Initialize the SummaryWriter
writer = SummaryWriter(comet_config={"disabled": False})

Comet will create an Experiment object under the hood for logging and automatically log all supported data types directly from the SummaryWriter. 

In the snippets below, we will go over the supported data types and how to log them. 

## Logging Scalars

Log Scalars and Scalar groups as you normally would in TensorboardX. Comet will autolog these scalar values as Metrics in the Comet UI. You can view logged metrics in the Metrics Tab, and Visualize them in the Charts Tab.

In [ ]:
def loss_func(x, alpha=0.75726054, beta=1.83646455, kappa=1.46651155, delta=0.67240758):
    return alpha - (alpha - beta) / (1.0 + (kappa * x) ** delta)


for n_iter in range(100):
    # data grouping by `slash`
    writer.add_scalar("data/loss", loss_func(n_iter), n_iter)

    # Change the displayed named of the metric
    writer.add_scalar(
        "data/scalar_custom",
        0.5 * loss_func(n_iter),
        n_iter,
        walltime=n_iter,
        display_name="my_custom_loss",
    )

    # Add scalar groups
    writer.add_scalars(
        "data/losses",
        {
            "train_loss": 1.5 * loss_func(n_iter),
            "val_loss": 1.75 * loss_func(n_iter),
            "test_loss": 2 * loss_func(n_iter),
        },
        n_iter,
    )

View the logged metrics in the Metrics tab in the Comet UI 

In [ ]:
display("metrics")

Visualize the logged metrics in the Charts tab in the Comet UI

In [ ]:
display("charts")

## Logging Hyperparameters

In [ ]:
hparams = {
    "lr": 0.1,
    "batch_size": 32,
    "n_hidden": 200,
    "bn": True,
}

writer.add_hparams(hparam_dict=hparams, metric_dict={})

You can view the logged parameters in the Parameters Tab in the Comet UI

In [ ]:
display("parameters")

## Logging Histograms

Log your model weights, gradients as Histograms. Comet has a dedicated 'Histograms' Tab where you can view 

In [ ]:
resnet18 = models.resnet18(False)
for iter in range(5):
    for name, param in resnet18.named_parameters():
        if "bn" not in name:
            writer.add_histogram(name, param, iter)

In [ ]:
display("histograms")

## Logging the Model Graph

Currently, Comet only supports logging ONNX graphs from TensorboardX.  

In [ ]:
%pip install onnx

In [ ]:
import torch.nn as nn

dummy_input = (torch.zeros(1, 3),)


class MultipleOutput(nn.Module):
    def __init__(self):
        super(MultipleOutput, self).__init__()
        self.Linear_1 = nn.Linear(3, 5)
        self.Linear_2 = nn.Linear(3, 7)

    def forward(self, x):
        return self.Linear_1(x), self.Linear_2(x)


model = MultipleOutput()

model_filename = "my-model.onnx"
torch.onnx.export(
    model,  # model being run
    dummy_input,  # model input (or a tuple for multiple inputs)
    model_filename,  # where to save the model (can be a file or file-like object)
    export_params=True,  # store the trained parameter weights inside the model file
    opset_version=10,  # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names=["input"],  # the model's input names
    output_names=["output"],  # the model's output names
    dynamic_axes={
        "input": {0: "batch_size"},  # variable length axes
        "output": {0: "batch_size"},
    },
)

writer.add_onnx_graph(model_filename)

The model graph will be logged as an asset. You can view it in the `others` directory in the Assets Tab of an Experiment. 

In [ ]:
display("assets")

## Logging Embeddings

Embeddings will be logged as Experiment Assets. They can be visualized using the Comet [Embeddings Projector](https://www.comet.ml/docs/user-interface/embeddings/)

In [ ]:
dataset = datasets.MNIST("mnist", train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]
features = images.view(100, 784)

writer.add_embedding(
    features, metadata=label, label_img=images.unsqueeze(1), tag="my_embedding"
)

Embeddings will be logged under the Assets Tab in an `embeddings` subdirectory. Click on the  

In [ ]:
display("assetStorage")

## Logging Figures

Comet will log Matplotlib Figure objects as Images 

In [ ]:
import matplotlib.pyplot as plt

plt.switch_backend("agg")

fig = plt.figure()

c1 = plt.Circle((0.2, 0.5), 0.2, color="r")
c2 = plt.Circle((0.8, 0.5), 0.2, color="r")

ax = plt.gca()
ax.add_patch(c1)
ax.add_patch(c2)
plt.axis("scaled")

writer.add_figure("matplotlib", fig)

In [ ]:
display("images")

## Logging Images


In [ ]:
# Logging an Image

x = torch.rand(32, 3, 64, 64)
x = vutils.make_grid(x, normalize=True, scale_each=True)

writer.add_image("Image", x, n_iter)  # Tensor

In [ ]:
# Logging Images with Bounding Boxes

writer.add_image_with_boxes(
    "imagebox_label",
    torch.ones(3, 240, 240) * 0.5,
    torch.Tensor([[10, 10, 100, 100], [101, 101, 200, 200]]),
    1,
    labels=["abcde", "fgh"],
)

In [ ]:
display(tab="images")

## Logging Audio

Logging audio requires the installation of an additional Python library `soundfile` and an additional system library `libsnd`. See the official instructions [how to install it](https://pypi.org/project/soundfile/#installation) and uncomment the following lines to logs Audio files.

In [ ]:
# %pip install soundfile scipy

In [ ]:
# sample_rate = 44100
# freq = 440

# x = torch.zeros(sample_rate * 2)
# for i in range(x.size(0)):
#     # sound amplitude should in [-1, 1]
#     x[i] = np.cos(freq * np.pi * float(i) / float(sample_rate))
# writer.add_audio("my-audio", x, 1)

In [ ]:
# display("audio")

## Logging Text

In [ ]:
writer.add_text("Text", "Lorem ipsum dolor sit amet")

Text will be logged under the Text Tab in the Comet UI

In [ ]:
display("text")

## Logging Curves

In [ ]:
labels = np.random.randint(2, size=100)
predictions = np.random.rand(100)

writer.add_pr_curve("pr-curve", labels, predictions, global_step=1)

Curves will be logged to the `Assets and Artifacts` tab under the `others` directory. 

In [ ]:
display("assetStorage")

## Close Logging to Comet

In [ ]:
writer.close()